<a href="https://colab.research.google.com/github/vsolodkyi/RecSystem_SkillBox/blob/main/module_1/Skillbox_recsys_11_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательная система на библиотеке surprise

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise```
или
```
$ conda install -c conda-forge scikit-surprise```

In [35]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [36]:
df

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [5]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [34]:
df_unpivot.tail()

,userID,itemID,rating
41,2,F,2.0
42,3,F,5.0
45,6,F,5.0
46,7,F,3.0
47,8,F,4.0


In [6]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [8]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633967 sha256=f5107083fe82c3514a5297db217b044966d3112ba7473957af0e5714e7f947c4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [9]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [12]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [13]:
testset[0:10]

[(1, 'F', 3.4642857142857144),
 (2, 'E', 3.4642857142857144),
 (5, 'B', 3.4642857142857144),
 (5, 'F', 3.4642857142857144),
 (7, 'B', 3.4642857142857144),
 (7, 'C', 3.4642857142857144),
 (7, 'E', 3.4642857142857144),
 (3, 'A', 3.4642857142857144),
 (3, 'E', 3.4642857142857144),
 (6, 'A', 3.4642857142857144)]

In [14]:
samples = [[0., 0., 0.], [0., .5, 0.], [1., 1., .5], [0.3, .5, 0.2], [.2, 1., .5]]
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(samples)
print(knn.kneighbors([[1., 1., 1.]]))

(array([[0.5       , 0.94339811]]), array([[2, 4]]))


In [15]:
print(knn.kneighbors([[1., 0., 1.]]))

(array([[1.11803399, 1.17473401]]), array([[2, 3]]))


In [16]:
from surprise import KNNBaseline
algo = KNNBaseline(k=1)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [17]:
predictions[0:5]

[Prediction(uid=1, iid='F', r_ui=3.4642857142857144, est=1.9577925900897966, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=2, iid='E', r_ui=3.4642857142857144, est=1.0422074099102034, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='B', r_ui=3.4642857142857144, est=4.082661845573158, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='F', r_ui=3.4642857142857144, est=2.0826618455731576, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=7, iid='B', r_ui=3.4642857142857144, est=3.8701641215302014, details={'actual_k': 1, 'was_impossible': False})]

In [18]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [19]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,1.957793
2,3.000000,4.000000,2.000000,4.000000,1.042207,2.000000
3,3.311189,2.000000,5.000000,5.000000,4.090446,5.000000
4,3.072513,4.197382,2.072513,4.072513,4.000000,3.976639
5,3.000000,4.082662,2.000000,4.000000,4.000000,2.082662
6,3.469084,5.000000,5.000000,5.000000,4.248341,5.000000
7,1.000000,3.870164,1.870164,2.000000,3.691628,3.000000
8,3.095874,1.909554,2.095874,4.095874,4.000000,4.000000


In [20]:
algo = KNNBaseline(k=3)
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [21]:
df_unpivot3 = df_unpivot.copy()
for i in predictions:
    df_unpivot3 = df_unpivot3.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot3.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.507435
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,3.256100,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.834977,2.000000,4.000000,4.000000,3.013312
6,3.418235,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.366011,1.857717,2.000000,3.095321,3.000000
8,2.579863,3.471928,3.463240,4.463240,4.000000,4.000000


## Кросс-валидация

In [22]:
from surprise.model_selection import cross_validate

In [23]:
cross_validate(algo, data, cv=2, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.4904  1.4338  1.4621  0.0283  
MAE (testset)     1.1604  1.1508  1.1556  0.0048  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([1.49041444, 1.43384912]),
 'test_mae': array([1.16037925, 1.15082401]),
 'fit_time': (0.004198312759399414, 0.00043654441833496094),
 'test_time': (0.00032448768615722656, 0.00022840499877929688)}

In [24]:
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.31347078, 1.7539855 , 1.55356241]),
 'fit_time': (0.006487607955932617,
  0.002483844757080078,
  0.001737833023071289),
 'test_time': (0.0002818107604980469,
  0.0001697540283203125,
  0.0002071857452392578)}

In [25]:
for i in range(1,6):
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.7188704727697548
2NN: 1.5656225424813293
3NN: 1.5323041629069671
4NN: 1.3335621381894345
5NN: 1.762407215082401


## Item-based подход

In [26]:
algo = KNNBaseline(k=5, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5 = df_unpivot.copy()
for i in predictions:
    df_unpivot5 = df_unpivot5.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.810335
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,2.859638,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.918340,2.000000,4.000000,4.000000,3.239815
6,3.053741,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.566434,2.632849,2.000000,3.095321,3.000000
8,2.615467,3.529529,3.297505,4.043101,4.000000,4.000000


In [27]:
sim_options = {'user_based': False } # Item-based подход

In [28]:
algo = KNNBaseline(k=5,sim_options=sim_options, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_items = df_unpivot.copy()
for i in predictions:
    df_unpivot5_items = df_unpivot5_items.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_items.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.329176
2,3.000000,4.000000,2.000000,4.000000,2.376574,2.000000
3,3.858465,2.000000,5.000000,5.000000,4.678967,5.000000
4,3.818825,4.122894,3.956402,4.261502,4.000000,4.155852
5,3.000000,3.337475,2.000000,4.000000,4.000000,3.311002
6,4.698465,5.000000,5.000000,5.000000,4.870350,5.000000
7,1.000000,1.789695,2.155844,2.000000,2.383400,3.000000
8,3.740899,4.021251,3.835183,4.150897,4.000000,4.000000


## Косинусная-мера

In [29]:
algo = KNNBaseline(k=5,sim_options= {'name': 'cosine'}, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_cos = df_unpivot.copy()
for i in predictions:
    df_unpivot5_cos = df_unpivot5_cos.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_cos.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.572362
2,3.000000,4.000000,2.000000,4.000000,2.927003,2.000000
3,2.754335,2.000000,5.000000,5.000000,3.270544,5.000000
4,3.134947,4.197382,2.134947,4.134947,4.000000,3.976639
5,3.000000,3.668772,2.000000,4.000000,4.000000,3.714426
6,2.939492,5.000000,5.000000,5.000000,3.360864,5.000000
7,1.000000,3.442211,2.951617,2.000000,2.802029,3.000000
8,2.700881,3.765123,3.231273,4.431273,4.000000,4.000000


## Datalens

В библиотеке есть встроенные Dataset. Можно воспользоваться ими.

Возьмем Dataset movielens (Побробнее о нем https://grouplens.org/datasets/movielens/ )

In [30]:
data = Dataset.load_builtin('ml-100k')#загружем данные о фильмах

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [31]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [32]:
df['user'].nunique(), df['item'].nunique()

(943, 1682)

In [33]:
for i in [1,3,5,7,20]:
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.2392253775378614
3NN: 1.039532750125179
5NN: 0.992372785470991
7NN: 0.974740701160591
20NN: 0.9391563315602175


## Задание:
Все задание выполняется на датасэтет 'ml-100k' (Dataset.load_builtin('ml-100k'))
1. Найдте для фильма алгоритмом kNN(k=20) с id 181 все неизвестные оценки (user-based). В ответе: посчитайте среднее полученных оценок.
2. Для параметров k = [10,20,30,40] найдите лучший item-based алгоритм (по метрике RMSE, кросс-валидация на 3 фолда).
Ответ: укажите k
3. Для лучшего алгоритма из пункта 2 найдте для фильма с id 181 все неизвестные оценки. В ответе: посчитайте среднее полученных оценок.

In [46]:
data = Dataset.load_builtin('ml-100k')#download data of films
train = data.build_full_trainset()
test = train.build_anti_testset()

In [47]:
sim_options = {'user_based': True }
knn_hw1 = KNNBaseline(k=20,sim_options= sim_options, verbose=False)
preds = knn_hw1.fit(train).test(test)

In [50]:
preds[:5]

[Prediction(uid='196', iid='302', r_ui=3.52986, est=3.9685439226477213, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.272678464142385, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.6080975397143273, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='196', iid='346', r_ui=3.52986, est=3.083782092665235, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='196', iid='474', r_ui=3.52986, est=4.4347543630733774, details={'actual_k': 20, 'was_impossible': False})]

In [58]:

df_hw1_1 = pd.DataFrame(columns=['userId', 'itemId', 'rating'])

In [59]:
for pred in preds:
  if int(pred.iid) == 181:
    df_hw1_1 = df_hw1_1.append({'userId': pred.uid, 'itemId': pred.iid, 'rating': pred.est}, ignore_index=True)
   

In [60]:
df_hw1_1.head()

,userId,itemId,rating
0,196,181,3.555154
1,186,181,3.991965
2,166,181,4.377213
3,253,181,4.196213
4,6,181,3.730430


In [62]:
df_hw1_1['rating'].mean()

3.952208342677188

## Для параметров k = [10,20,30,40] найдите лучший item-based алгоритм (по метрике RMSE, кросс-валидация на 3 фолда). Ответ: укажите k

In [66]:
best_algo = {'i': None, 'accuracy': 5}
for i in [10,20,30,40]:
    sim_options = {'user_based': False }
    knn_hw2 = KNNBaseline(k=i, verbose=False, sim_options=sim_options) #отключим вывод логирования
    cv=cross_validate(knn_hw2, data, measures=['RMSE'], cv=3, verbose=False)
    if np.mean(cv['test_rmse']) < best_algo['accuracy']:
      best_algo['accuracy'] = np.mean(cv['test_rmse'])
      best_algo['i'] = i



In [67]:
print('best algorythm is at {0} k and mean RMSE = {1}'.format(best_algo['i'], best_algo['accuracy']))

best algorythm is at 40 k and mean RMSE = 0.9389114464329936


## Для лучшего алгоритма из пункта 2 найдте для фильма с id 181 все неизвестные оценки. В ответе: посчитайте среднее полученных оценок.

In [79]:
df_hw1_3 = pd.DataFrame(columns=['userId', 'itemId', 'rating'])

In [72]:
knn_hw1_3 = KNNBaseline(k=40, verbose=False, sim_options=sim_options) #отключим вывод логирования

In [73]:
preds = knn_hw1_3.fit(train).test(test)

In [80]:
for pred in preds:
  if int(pred.iid) == 181:
    df_hw1_3 = df_hw1_3.append({'userId': pred.uid, 'itemId': pred.iid, 'rating': pred.est}, ignore_index=True)

In [81]:
df_hw1_3['rating'].mean()

4.033326822883139